In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import wandb

# Set Seed
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(42)

# Check Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {DEVICE}")

# Login to WandB
# wandb.login(key="YOUR_KEY_HERE")

In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm

BIDS_ROOT = '/ocean/projects/cis250019p/gandotra/11785-gp-eeg/ds005589'
IMAGE_DIR = '/ocean/projects/cis250019p/gandotra/11785-gp-eeg/images'
CAPTIONS_FILE = '/ocean/projects/cis250019p/gandotra/11785-gp-eeg/captions.txt'
ALL_SUBJECTS = ['sub-02', 'sub-03', 'sub-05', 'sub-09', 'sub-14', 'sub-15', 
                'sub-17', 'sub-19', 'sub-20', 'sub-23', 'sub-24', 'sub-28', 'sub-29']
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class EEG_Paper_Replication_Dataset(Dataset):
    def __init__(self, bids_root, images_dir, captions_path, 
                 subject_list, session_list, 
                 clamp_thres=500, stats=None):
        
        self.bids_root = bids_root
        self.images_dir = images_dir
        self.clamp_thres = clamp_thres
        self.trial_metadata = []
        
        # --- Create Subject Mapping ---
        # Map 'sub-02' -> 0, 'sub-03' -> 1, etc.
        # We sort to ensure consistency across Train/Val/Test sets
        self.subject_to_idx = {sub: i for i, sub in enumerate(sorted(list(set(subject_list))))}
        
        # 1. Load Captions Helper
        self.captions_dict = self._load_captions(captions_path)
        self.category_to_idx = {cat: i for i, cat in enumerate(sorted(set(c for c, _ in self.captions_dict.values())))}
        
        # 2. Scan Metadata
        print(f"Scanning metadata for {session_list}...")
        for sub in subject_list:
            for ses in session_list:
                for run in ['01', '02', '03', '04']:
                    session_path = os.path.join(self.bids_root, sub, ses)
                    csv_path = os.path.join(session_path, f"{sub}_{ses}_task-lowSpeed_run-{run}_image.csv")
                    npy_path = os.path.join(session_path, f"{sub}_{ses}_task-lowSpeed_run-{run}_1000Hz.npy")
                    
                    if not (os.path.exists(csv_path) and os.path.exists(npy_path)):
                        continue
                        
                    try:
                        csv_data = pd.read_csv(csv_path)
                        for i, row in csv_data.iterrows():
                            img_base_name = self._get_base_name(row['FilePath'])
                            if not img_base_name: continue
                            
                            category, caption = self.captions_dict.get(img_base_name, (None, None))
                            if not category: continue
                            
                            self.trial_metadata.append({
                                'npy_path': npy_path,
                                'trial_index': i,
                                'label': self.category_to_idx[category],
                                'subject_id': self.subject_to_idx[sub] 
                            })
                    except:
                        continue
        
        print(f"Found {len(self.trial_metadata)} trials.")

        # 3. Compute Global Statistics (GFS)
        if stats is None:
            print("Computing Global Statistics (this takes ~1 min)...")
            self.mean, self.std = self._compute_global_stats()
        else:
            self.mean, self.std = stats

    def _compute_global_stats(self):
        subset_indices = range(0, len(self.trial_metadata), 10)
        sum_x = 0
        sum_sq_x = 0
        count = 0
        
        for i in tqdm(subset_indices, desc="Calculating Stats"):
            meta = self.trial_metadata[i]
            d = np.load(meta['npy_path'])[meta['trial_index']]
            d = np.clip(d, -self.clamp_thres, self.clamp_thres)
            
            sum_x += np.mean(d)
            sum_sq_x += np.mean(d**2)
            count += 1
            
        global_mean = sum_x / count
        global_std = np.sqrt((sum_sq_x / count) - (global_mean**2))
        return float(global_mean), float(global_std)

    def get_stats(self): return self.mean, self.std

    def _load_captions(self, path):
        d = {}
        with open(path, 'r') as f:
            next(f)
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 4: d[parts[2]] = (parts[1], parts[3])
        return d

    def _get_base_name(self, p):
        try:
            bn = os.path.splitext(os.path.basename(str(p).replace('\\', '/')))[0]
            if bn.endswith('_resized'): return bn[:-8]
            return bn
        except: return None

    def __len__(self): return len(self.trial_metadata)

    def __getitem__(self, idx):
        meta = self.trial_metadata[idx]
        eeg_data = np.load(meta['npy_path'])[meta['trial_index']]
        eeg_data = np.clip(eeg_data, -self.clamp_thres, self.clamp_thres)
        
        # Global Feature Standardization
        eeg_data = (eeg_data - self.mean) / (self.std + 1e-6)
        
        return (torch.tensor(eeg_data, dtype=torch.float32), 
                torch.tensor(meta['label'], dtype=torch.long),
                torch.tensor(meta['subject_id'], dtype=torch.long))

In [ ]:
# 1. Create Transforms (Standard ImageNet stats)
img_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 2. Instantiate Training Dataset
print("--- Init Train ---")
train_ds = EEG_Paper_Replication_Dataset(
    bids_root=BIDS_ROOT, 
    images_dir=IMAGE_DIR, 
    captions_path=CAPTIONS_FILE, 
    subject_list=ALL_SUBJECTS, 
    session_list=['ses-01', 'ses-02', 'ses-03'], 
    clamp_thres=500
)
# Save stats to use for validation (Prevent data leakage)
stats = train_ds.get_stats()

# 3. Instantiate Validation Dataset
print("\n--- Init Validation ---")
val_ds = EEG_Paper_Replication_Dataset(
    bids_root=BIDS_ROOT, 
    images_dir=IMAGE_DIR, 
    captions_path=CAPTIONS_FILE, 
    subject_list=ALL_SUBJECTS, 
    session_list=['ses-04'], 
    clamp_thres=500,
    stats=stats # <--- IMPORTANT: Use training stats
)

# 4. Define The Loaders
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

print(f"\n✅ Loaders Ready: {len(train_loader)} training batches, {len(val_loader)} validation batches.")

In [ ]:
class EEG_ViT_1D(nn.Module):
    def __init__(self, num_subjects=13, num_classes=20):
        super().__init__()
        
        # --- Hyperparameters ---
        self.patch_size = 50      # 50ms window 
        self.stride = 25          # 50% overlap
        self.embed_dim = 128      # Feature size
        self.num_heads = 4
        self.depth = 2
        
        # --- 1. Tokenizer (The "Patchify" Step) ---
        # Input: (Batch, 122, 500) -> Output: (Batch, 128, ~19)
        self.tokenizer = nn.Sequential(
            nn.Conv1d(122, self.embed_dim, kernel_size=self.patch_size, stride=self.stride, padding=0),
            nn.BatchNorm1d(self.embed_dim),
            nn.LeakyReLU(0.1)
        )
        
        # --- 2. Learnable "Class Token" ---
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embed_dim))
        
        # --- 3. Positional Embedding ---
        self.pos_embedding = nn.Parameter(torch.randn(1, 32, self.embed_dim) * 0.01)
        
        # --- 4. Transformer Encoder ---
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embed_dim, 
            nhead=self.num_heads, 
            dim_feedforward=512, 
            dropout=0.5, 
            batch_first=True,
            norm_first=True 
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=self.depth)
        
        # --- 5. Subject-Specific Heads ---
        self.heads = nn.ModuleList([
            nn.Linear(self.embed_dim, num_classes) for _ in range(num_subjects)
        ])

    def forward(self, x, subject_ids):
        # Input comes in as (Batch, 500, 122) -> We need (Batch, 122, 500)
        if x.shape[1] == 500 and x.shape[2] == 122:
            x = x.permute(0, 2, 1)
            
        # Safety Crop (in case data is >500)
        if x.shape[2] > 500:
            x = x[:, :, :500]
            
        # ----------------------------------------
        
        # 1. Tokenize
        x = self.tokenizer(x)     # Output: (Batch, 128, 19)
        x = x.permute(0, 2, 1)    # Output: (Batch, 19, 128) -> (Batch, Seq, Dim)
        
        b, seq_len, _ = x.shape
        
        # 2. Append CLS Token
        cls_tokens = self.cls_token.expand(b, -1, -1) 
        x = torch.cat((cls_tokens, x), dim=1) # (Batch, 20, 128)
        
        # 3. Add Positional Embedding
        x = x + self.pos_embedding[:, :seq_len + 1, :]
        
        # 4. Transformer Attention
        x = self.transformer(x)
        
        # 5. Extract ONLY the CLS token output
        cls_output = x[:, 0, :]   # (Batch, 128)
        
        # 6. Subject Routing
        logits = torch.zeros(x.shape[0], 20).to(x.device)
        unique_subs = torch.unique(subject_ids)
        
        for sub in unique_subs:
            mask = (subject_ids == sub)
            logits[mask] = self.heads[sub.long()](cls_output[mask])
            
        return logits

In [ ]:
RESUME_TRAINING = False
CHECKPOINT_PATH = "best_model_vit_1d.pth"
TOTAL_EPOCHS = 100

wandb.init(
    project="eeg-classification",
    name="vit-1d-run",
    config={
        "architecture": "ViT-1D",
        "dataset": "GFS",
        "epochs": TOTAL_EPOCHS,
        "lr": 1e-3
    }
)

model = EEG_ViT_1D(num_subjects=13, num_classes=20).to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-3)
criterion = nn.CrossEntropyLoss()

print(f"Model Parameters: {sum(p.numel() for p in model.parameters()):,}")

start_epoch = 1
best_val_acc = 0.0

if RESUME_TRAINING and os.path.exists(CHECKPOINT_PATH):
    print(f"🔄 Attempting to resume from {CHECKPOINT_PATH}...")
    try:
        checkpoint = torch.load(CHECKPOINT_PATH, map_location=DEVICE)
        
        if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'])
            if 'optimizer_state_dict' in checkpoint:
                optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint.get('epoch', 0) + 1
            best_val_acc = checkpoint.get('val_acc', 0.0)
            print(f"✅ Resuming from Epoch {start_epoch} with Best Acc {best_val_acc:.2f}%")
        else:
            model.load_state_dict(checkpoint)
            print(f"⚠️ Only weights found. Resuming from Epoch 1.")
            
    except Exception as e:
        print(f"❌ Error loading checkpoint: {e}. Starting fresh.")
else:
    print("🆕 Starting a fresh training run.")

print(f"\n🚀 Training from Epoch {start_epoch} to {TOTAL_EPOCHS}...")

for epoch in range(start_epoch, TOTAL_EPOCHS + 1):
    model.train()
    correct = 0
    total = 0
    
    for eeg, label, sub_id in tqdm(train_loader, desc=f"Epoch {epoch}", leave=False):
        eeg, label, sub_id = eeg.to(DEVICE), label.to(DEVICE), sub_id.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(eeg, sub_id)
        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        _, pred = outputs.max(1)
        correct += pred.eq(label).sum().item()
        total += label.size(0)
        
    train_acc = 100. * correct / total
    
    model.eval()
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for eeg, label, sub_id in val_loader:
            eeg, label, sub_id = eeg.to(DEVICE), label.to(DEVICE), sub_id.to(DEVICE)
            outputs = model(eeg, sub_id)
            _, pred = outputs.max(1)
            val_correct += pred.eq(label).sum().item()
            val_total += label.size(0)
            
    val_acc = 100. * val_correct / val_total
    
    print(f"Epoch {epoch:02d}: Train Acc {train_acc:.2f}% | Val Acc {val_acc:.2f}%")
    
    wandb.log({
        "epoch": epoch,
        "train_acc": train_acc,
        "val_acc": val_acc,
        "train_loss": loss.item()
    })
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'train_acc': train_acc
        }, CHECKPOINT_PATH)
        
        wandb.save(CHECKPOINT_PATH) 
        
        print(f"  ✅ Best Model Saved & Uploaded! ({val_acc:.2f}%)")

wandb.finish()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score

print("Loading Test Set (Session 5)...")
test_ds = EEG_Paper_Replication_Dataset(
    BIDS_ROOT, IMAGE_DIR, CAPTIONS_FILE, ALL_SUBJECTS, 
    ['ses-05'], # FINAL TEST SET
    stats=stats,
    clamp_thres=500
)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False, num_workers=4)

def evaluate_detailed(model, loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    subject_results = {i: {'correct': 0, 'total': 0} for i in range(len(ALL_SUBJECTS))}
    
    with torch.no_grad():
        for eeg, label, sub_id in tqdm(loader, desc="Testing"):
            eeg, label, sub_id = eeg.to(device), label.to(device), sub_id.to(device)
            
            outputs = model(eeg, sub_id)
            _, preds = outputs.max(1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(label.cpu().numpy())
            
            for i in range(len(label)):
                sid = sub_id[i].item()
                is_correct = (preds[i] == label[i]).item()
                subject_results[sid]['total'] += 1
                subject_results[sid]['correct'] += is_correct
                
    return all_labels, all_preds, subject_results

print("Running Final Evaluation...")
y_true, y_pred, sub_metrics = evaluate_detailed(model, test_loader, DEVICE)

overall_acc = accuracy_score(y_true, y_pred) * 100
print(f"\n🏆 Final Test Accuracy: {overall_acc:.2f}%")

sub_accs = []
sub_names = []
for sid, metrics in sub_metrics.items():
    if metrics['total'] > 0:
        acc = (metrics['correct'] / metrics['total']) * 100
        sub_accs.append(acc)
        sub_names.append(ALL_SUBJECTS[sid])
        
plt.figure(figsize=(12, 5))
sns.barplot(x=sub_names, y=sub_accs, palette="viridis")
plt.axhline(y=overall_acc, color='r', linestyle='--', label=f'Avg: {overall_acc:.1f}%')
plt.axhline(y=5.0, color='gray', linestyle='--', label='Random Chance (5%)')
plt.title("Per-Subject Classification Accuracy (Session 5)")
plt.ylabel("Accuracy (%)")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

cat_map = {v: k for k, v in test_ds.category_to_idx.items()}
cat_names = [cat_map[i] for i in range(len(cat_map))]

cm = confusion_matrix(y_true, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

plt.figure(figsize=(14, 12))
sns.heatmap(cm_norm, annot=False, fmt=".2f", cmap="Blues", xticklabels=cat_names, yticklabels=cat_names)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()